In [ ]:
#C:\Users\90536\Downloads\off100.PNG
import cv2
import numpy as np

mouse_coordinates = []
pitch_points = []
image_points = []

#It fetchs/gives Mouse_corrdinates when we apply an Event of "Clicking the Left Button Down"
def get_mouse_coordinates(Event, x, y, flags, params):
    if Event == cv2.EVENT_LBUTTONDOWN:
        mouse_coordinates.append([x, y])

#To get the offender and defender positions or to select any two points [this function helps to do it] 
def fetch_points(Img, NumberofPoints):

    while 1 and len(mouse_coordinates) < NumberofPoints:
    
        cv2.namedWindow('Player Coordinates')  #To create a new Window
        cv2.setMouseCallback('Player Coordinates', get_mouse_coordinates, param=Img) #Call Back function to get the Mouse Point's x and y
        
        cv2.imshow('Player Coordinates', Img)	#To show the Image
        image_h, image_w, ch = Img.shape        #To get the Shape/Dimension of the Image
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(Img, 'Click any Two points to Find the OFFSIDE distance',
                (20, image_h-500), font, 1, (255, 0, 0), 2, cv2.LINE_AA)     #To put Text on the image 
        if cv2.waitKey(20) & 0xFF == 27: #Image Output waits until we press a button
            break

    
    cv2.destroyAllWindows() #All opened Windows Diappear
    return mouse_coordinates #To return the Output(Mouse Points Coordinates) 

#To draw a Line and Normal 
def find_LinePointsForPitch(pitch, output_points):
    line_points = []
    normals = []
    for p in output_points:
        x, y = p
        x = int(x)            
        y = int(y)
        line_points.append([x, 17]) 
        line_points.append([x, 584]) #These code lines help to find the Top and Bottom Points to the Transformed Pitch point .These two points help to form Line 
        normals.append([x, y, 0])
        normals.append([x, y, 10]) #To get the normal or perpendicular Line
        
    return line_points, normals 

#To Draw a Line on Image Plane 
def draw_line_on_ImagePlane(image, reverse_output_points):
    for i in range(0, len(reverse_output_points), 2):
        x1, y1 = reverse_output_points[i]
        x2, y2 = reverse_output_points[i + 1]

        x1 = int(x1)
        y1 = int(y1)
        x2 = int(x2)
        y2 = int(y2)
        #Till here,we have fetch the two Points to draw a Line 

        color = [0, 255,255] if i < 1 else [255, 0, 0] #To give any color to the Line
        
        cv2.line(image, (x1, y1), (x2, y2), color, 2)  #Function to draw a Line


def main():
    print('The User will input the Path of the Image'
          'It takes in 2 user input and measures the Distance b/w those selected points'
          'It also draws normal and parallel lines to field ground')

    pitch_path = './pitch.png'
    image = cv2.imread(input("Write file name with it's path: "))
    pitch = cv2.imread(pitch_path)
    #cv2.imread helps to Read the Image as specified the Given Path
    
    # We get two points by using fetch_points function which is defined/shown above
    user_points = np.array(fetch_points(image, 2), dtype='float32')
    cv2.destroyAllWindows()

    
    isMatchingsProper = False
    while not isMatchingsProper:
        #I have used cv2.setMouseCallback and cv2.EVENT_LBUTTONDOWN event inorder to SELECT the CORRESPONDING/MATCHING Points from Image and Field.
        #following points i got by using that function and these points will help to find the HOMOGRAPHY 
        MatchingImagePoints = [[869, 139], [843, 421], [762, 216], [857, 239], [853, 281], [861, 191], [160, 118], [238, 167], [80, 146], [325, 62], [129, 190]]
        MatchingFieldPoints = [[781, 134], [782, 469], [750, 300], [781, 303], [781, 361], [781, 240], [480, 225], [556, 301], [481, 300], [481, 18], [530, 358]]
        
        #To makesure We DIDN'T GAVE LESS THAN 4 POINTS and that Same number of Image and Filed Points are Selected we have applied this if Condition
        if len(MatchingImagePoints) > 3 and len(MatchingImagePoints) == len(MatchingFieldPoints):
            break
        global pitch_points, image_points
        pitch_points = []
        image_points = []
        

    # Get field Dimensions
    length = 120
    height = 90

   
    # Get Distance per Pixel
    width_per_pixel = length / 876
    height_per_pixel = height / 567

    # Convert python arrays to numpy arrays
    #Our variables are in List.To form an Matric like format ,so that we can apply Homography , we have converted it into an Numpy Array
    MatchingImagePoints = np.array(MatchingImagePoints)
    MatchingFieldPoints = np.array(MatchingFieldPoints)

    # Find Homography Parameters between given points of two Images
    #Homography Input: Takes input of corresponding or matching points from Field and the Image
    #Homography Output: 3 by 3 matrix which represent the Transformation Parameters (9 parameters).
    h, status = cv2.findHomography(MatchingImagePoints, MatchingFieldPoints)
    
    # Map user selected players' coordinates to pitch image
    input_coor = np.array([user_points])
    output = cv2.perspectiveTransform(input_coor, h)[0]
    #Inorder to TRANSFORM the points from Image to the Field.This functions helps to MAP the points from Image to it's corresponding points in Field.
    #Input : Points which we want to map are as an input and also the Homography Matrix is an input so that we can multiply them and get the corresponding Points in the Field.
    
    #Function defined above, it is not a predefined function.This function Finds points which help to draw parallel lines on Pitch	
    line_points, Normals = find_LinePointsForPitch(pitch, output)
    
    #We simply subtracted the x corrdinates of the two lines to find the Distance b/w them . Then Multiplied by width_per_pixel to get Distance in meter or cm units but not in pixel units
    Distance = np.absolute(line_points[0][0] - line_points[2][0]) * width_per_pixel
    
    # Get line points from pitch points
    PitchPoints_on_ImagePlane = cv2.perspectiveTransform(np.array([line_points], dtype='float32'), np.linalg.inv(h))[0]
    
    draw_line_on_ImagePlane(image, PitchPoints_on_ImagePlane)

    # Shows Image with lines and distance
    
    font = cv2.FONT_HERSHEY_COMPLEX
    image_h, image_w, ch = image.shape
       
    cv2.putText(image, 'DISTANCE: {0:.4f} metres'.format(Distance),
                (200, image_h-225), font, 1, (0, 0, 255), 2, cv2.LINE_AA)
    cv2.imshow('Output', image)
    
    #If ANY Key is Pressed then ALL image Windows will Disappear
    if cv2.waitKey(0):
        cv2.destroyAllWindows()




if __name__ == '__main__':
    main()


The User will input the Path of the ImageIt takes in 2 user input and measures the Distance b/w those selected pointsIt also draws normal and parallel lines to field ground
